In [ ]:
import uproot
import json
import numpy as np
import pandas as pd
from vis_functions import *
from timestamp_calc import *
from collect_data import *

notebook_type = "src_vs_bkg_charge"
params_file = open("/home/juno/arsgav/params.json")
PARAMS = json.load(params_file)
PARAMS['type'] = notebook_type

neptune_run = False
if neptune_run:
    run = neptune.init_run(
        project="legnaro/LegnaroSetup",
        api_token=PARAMS['api_token'],
        tags=[
            PARAMS['path'],
            PARAMS['run_numbers'],
            PARAMS['source_names'],
            PARAMS['output_file'],
            notebook_type,
        ],
        name="LNL",
        # with_id="LNL-"
    )

    del PARAMS['api_token']
    run['PARAMS'] = PARAMS

In [ ]:
path = PARAMS['path']
output_file = PARAMS['output_file']
run_numbers = PARAMS['run_numbers'].split(', ')
source_names = PARAMS['source_names'].split(', ')
src_bkg_run_numbers = PARAMS['src_bkg_run_numbers'].split(', ')
src_bkg_names = PARAMS['src_bkg_names'].split(', ')
graph_height = int(PARAMS['graph_height'])
graph_width = int(PARAMS['graph_width'])
left_shift = int(PARAMS['left_shift'])

In [ ]:
paths = [f'{path}run{i}/tier1/{output_file}' for i in run_numbers]
print("Paths: ")
[print("  ", el) for el in paths]

total_charges = join_by_run(paths, ['total_charge'])
if neptune_run:
    plot_distrs_one_param(total_charges, 'total_charge', source_names, "Total charge",
                          bkg_subtract=False, Nbins=np.linspace(0, 1e-4, 500), bar_step_x_shift=0.475,
                          range_x=[0, 1e-4], line_width=0.0, opacity=0.7, return_values=False,
                          neptune_run=neptune_run, run=run, run_plot_name=f"Total charges for {PARAMS['run_numbers']} runs")
else:
    plot_distrs_one_param(total_charges, 'total_charge', source_names, "Total charge",
                          bkg_subtract=False, Nbins=np.linspace(0, 1e-4, 500), bar_step_x_shift=0.475,
                          range_x=[0, 1e-4], line_width=0.0, opacity=0.7, return_values=False,)

In [ ]:
for i in run_numbers:
    print(f"-_____________________run number {i}_____________________")
    timestamp_list = timestamp_calc(path, run_number=i, dir_name="tier1", left_shift=left_shift)
    print(timestamp_list)
    print(f"**************Number of channels: {len(timestamp_list)}")
    print("\n \n")

In [ ]:
paths = [f'{path}run{i}/tier1/{output_file}' for i in src_bkg_run_numbers]
print("Paths: ")
[print("  ", el) for el in paths]

total_charges = join_by_run(paths, ['total_charge'])
colors = ['royalblue', 'darkred']

bkg_time = np.median(timestamp_calc(path, src_bkg_run_numbers[0], 'tier1', left_shift=left_shift))
cs_time = np.median(timestamp_calc(path, src_bkg_run_numbers[1], 'tier1', left_shift=left_shift))

scale_factor = cs_time / bkg_time

if neptune_run:
    bins, srs_values, bkg_values = plot_distrs_one_param(total_charges, 'total_charge', src_bkg_names, "Total charge",
                                  colors_flag=True, bkg_subtract=False, Nbins=np.linspace(0, 1e-4, 500),
                                  range_x=[0, 1e-4], line_width=0.0, opacity=0.7, return_values=True, bar_step_x_shift=0.6,
                                  bkg_n=0, scale_factor=scale_factor, left_shift=left_shift, colors=colors,
                                  neptune_run=neptune_run, run=run, run_plot_name=f"Total charges for {PARAMS['src_bkg_names']}.")

    
    plot_distrs_one_param(total_charges, 'total_charge', src_bkg_names, "Total charge", colors_flag=True,
                              bkg_subtract=True, Nbins=np.linspace(0, 1e-4, 500), bar_step_x_shift=0.6,
                              range_x=[0, 1e-4], line_width=0.0, opacity=0.7, return_values=False,
                              bkg_n=0, scale_factor=scale_factor, left_shift=left_shift, colors=colors,
                              neptune_run=neptune_run, run=run, run_plot_name=f"Total charge for {src_bkg_names[1]}. Background subtracted.")

    
else:
    bins, srs_values, bkg_values = plot_distrs_one_param(total_charges, 'total_charge', src_bkg_names, "Total charge",
                                  colors_flag=True, bkg_subtract=False, Nbins=np.linspace(0, 1e-4, 500), bar_step_x_shift=0.6,
                                  range_x=[0, 1e-4], line_width=0.0, opacity=0.7, return_values=True,
                                  bkg_n=0, scale_factor=scale_factor, left_shift=left_shift, colors=colors)
    
    plot_distrs_one_param(total_charges, 'total_charge', src_bkg_names, "Total charge", colors_flag=True,
                              bkg_subtract=True, Nbins=np.linspace(0, 1e-4, 500), bar_step_x_shift=0.6,
                              range_x=[0, 1e-4], line_width=0.0, opacity=0.7, return_values=False,
                              bkg_n=0, scale_factor=scale_factor, left_shift=left_shift, colors=colors)    

In [ ]:
# df = pd.DataFrame([bins, srs_values, bkg_values]).T
# df.columns = ['bins', 'srs_values', 'bkg_values']
# df.to_csv('Co60.csv', index=False)
# df

In [ ]:
paths = [f'{path}run{i}/tier1/{output_file}' for i in src_bkg_run_numbers]
print("Paths: ")
[print("  ", el) for el in paths]

charges = join_by_run(paths, ['charge'])

bkg_time = timestamp_calc(path, src_bkg_run_numbers[0], 'tier1', left_shift=left_shift)
cs_time = timestamp_calc(path, src_bkg_run_numbers[1], 'tier1', left_shift=left_shift)
timestamps = [bkg_time, cs_time]

if neptune_run:
    plot_charges_by_channels_src_and_bkg(charges, timestamps, src_bkg_names, width=graph_width, height=graph_height,
                                         Nbins=np.linspace(0, 5e-6, 50), bkg_subtract=False,
                                         neptune_run=neptune_run, run=run,
                                         run_plot_name=f"Charge by channel for {PARAMS['src_bkg_names']}.")

    plot_charges_by_channels_src_and_bkg(charges, timestamps, src_bkg_names, width=graph_width, height=graph_height,
                                     Nbins=np.linspace(0, 5e-6, 50), bkg_subtract=True,
                                     neptune_run=neptune_run, run=run,
                                     run_plot_name=f"Charge by channel for {src_bkg_names[1]}. Background subtracted.")
else:
    plot_charges_by_channels_src_and_bkg(charges, timestamps, src_bkg_names, width=graph_width, height=graph_height,
                                 Nbins=np.linspace(0, 5e-6, 50), bkg_subtract=False)

    plot_charges_by_channels_src_and_bkg(charges, timestamps, src_bkg_names, width=graph_width, height=graph_height,
                                     Nbins=np.linspace(0, 5e-6, 50), bkg_subtract=True)

In [ ]:
if neptune_run:
    run.stop()